# DeepSeek API 对话类

> 指导文章：[DeepSeek API 统一模型对话逻辑与流式输出](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/DeepSeek%20API%20统一模型对话逻辑与流式输出.md)


在线链接：[Kaggle](https://www.kaggle.com/code/aidemos/deepseek-api-guide-5) | [Colab](https://colab.research.google.com/drive/14u47q-lGfH7l1ehkBuTU0kgIsNarap9J?usp=sharing)

## 导入库

你需要先运行当前模块才能正确执行后续的代码。

In [1]:
import os
import ipywidgets as widgets
from IPython.display import display, HTML
from typing import Dict, Optional

from openai import OpenAI

## 配置设置

> 这里是一个快捷配置类，可以不用关注这个它的代码细节。

在接下来的学习中，都将用 `APIConfigManager` 进行配置，使用 `Shift + 回车` 运行后，填写对应平台的 `API Key` 后点击 `保存配置` 按钮，就可以顺利运行之后的代码。

后续切换平台或者模型后，也需要点击 `保存配置` 按钮。

In [2]:
class APIConfigManager:
    """API 配置管理器，用于处理环境变量和配置信息"""
    
    # 状态颜色映射
    STATUS_COLORS = {
        'success': 'green',
        'warning': 'orange',
        'error': 'red'
    }
    
    # 平台配置信息，新平台或配置可以在此处修改
    PLATFORM_CONFIG = {
        'DeepSeek': {
            'api_key_name': 'DEEPSEEK_API_KEY',
            'base_url': 'https://api.deepseek.com',
            'chat_model_id': 'deepseek-chat',
            'reasoner_model_id': 'deepseek-reasoner'
        },
        '硅基流动': {
            'api_key_name': 'SILICONFLOW_API_KEY',
            'base_url': 'https://api.siliconflow.cn/v1',
            'chat_model_id': 'deepseek-ai/DeepSeek-V3',
            'reasoner_model_id': 'deepseek-ai/DeepSeek-R1'
        },
        '阿里云百炼': {
            'api_key_name': 'DASHSCOPE_API_KEY',
            'base_url': 'https://dashscope.aliyuncs.com/compatible-mode/v1',
            'chat_model_id': 'deepseek-v3',
            'reasoner_model_id': 'deepseek-r1'
        },
        '百度智能云': {
            'api_key_name': 'BAIDU_API_KEY',
            'base_url': 'https://qianfan.baidubce.com/v2',
            'chat_model_id': 'deepseek-v3',
            'reasoner_model_id': 'deepseek-r1'
        },
        '字节火山引擎': {
            'api_key_name': 'ARK_API_KEY',
            'base_url': 'https://ark.cn-beijing.volces.com/api/v3',
            'chat_model_id': 'deepseek-v3-241226',
            'reasoner_model_id': 'deepseek-r1-250120'
        }
    }
    
    def __init__(self):
        """初始化配置管理器"""
        self.current_config: Dict[str, str] = {}
        self._setup_widgets()
        
    def _setup_widgets(self):
        """设置 ipywidgets 界面组件"""
        self.platform_dropdown = widgets.Dropdown(
            options=list(self.PLATFORM_CONFIG.keys()),
            description='选择平台:',
            layout={'width': '300px'}
        )
        
        self.model_type_dropdown = widgets.Dropdown(
            options=[('聊天模型', 'chat'), ('推理模型', 'reasoner')],
            description='模型类型:',
            layout={'width': '300px'}
        )

        # 默认允许编辑模型ID
        self.model_id_input = widgets.Text(
            description='模型 ID:',
            layout={'width': '500px'},
            disabled=False
        )
        
        self.api_key_input = widgets.Password(
            description='API Key:',
            layout={'width': '500px'}
        )
        
        self.status_label = widgets.HTML(
            value='',
            layout={'width': '400px'}
        )
        
        self.save_button = widgets.Button(
            description='保存配置',
            button_style='primary',  # 默认灰色，设置为蓝色
            layout={'width': '150px'}
        )
        
        # 绑定事件处理函数
        self.platform_dropdown.observe(self._on_platform_change, names='value')
        self.model_type_dropdown.observe(self._on_model_type_change, names='value')
        self.save_button.on_click(self._on_save_click)
        
        # 创建布局
        self.widget = widgets.VBox([
            self.platform_dropdown,
            self.model_type_dropdown,
            self.model_id_input,
            self.api_key_input,
            self.status_label,
            self.save_button
        ])
    
    def _on_platform_change(self, change):
        """处理平台选择变化事件"""
        if not change.new:
            return
            
        platform = change.new
        model_type = self.model_type_dropdown.value
        platform_config = self.PLATFORM_CONFIG[platform]
        
        # 更新 API Key 输入框描述和值
        self.api_key_input.description = f'API Key ({platform_config["api_key_name"]}):'
        api_key = os.getenv(platform_config['api_key_name'])
        self.api_key_input.value = api_key if api_key else ''
        
        # 获取当前模型 ID
        current_model_id = platform_config[f'{model_type}_model_id']
        
        # 设置模型ID输入框的描述信息
        self.model_id_input.description = '模型 ID:'
        
        # 尝试从环境变量获取模型ID（优先级高于平台配置）
        env_var_name = f"{platform_config['api_key_name'].replace('_API_KEY', '')}_{model_type.upper()}_MODEL_ID"
        env_model_id = os.getenv(env_var_name, '')
        
        # 如果环境变量中有值，使用环境变量的值，否则使用平台配置（PLATFORM_CONFIG）中的默认值
        self.model_id_input.value = env_model_id if env_model_id else current_model_id
        
        if api_key:
            self._update_status('检测到所需配置，点击"保存配置"按钮完成设置', 'success')
        else:
            self._update_status(f'请输入 {platform_config["api_key_name"]}，填写完成后点击"保存配置"按钮', 'warning')
    
    
    def _on_model_type_change(self, change):
        """处理模型类型变化事件"""
        if change.new and self.platform_dropdown.value:
            # 重用平台变更的逻辑
            self._on_platform_change(type('Change', (), {'new': self.platform_dropdown.value})())
    
    def _on_save_click(self, b):
        """处理保存按钮点击事件"""
        try:
            platform = self.platform_dropdown.value
            platform_config = self.PLATFORM_CONFIG[platform]
            model_type = self.model_type_dropdown.value
            
            if not platform or not self.api_key_input.value:
                raise ValueError('请填写所有必填项')
            
            if not self.model_id_input.value:
                raise ValueError('请填写模型 ID')
            
            # 设置API Key环境变量
            os.environ[platform_config['api_key_name']] = self.api_key_input.value
            
            # 使用用户输入的模型ID
            self.current_config = {
                'api_key': self.api_key_input.value,
                'base_url': platform_config['base_url'],
                'model': self.model_id_input.value
            }
            
            self._update_status('配置已保存', 'success')
        except Exception as e:
            self._update_status(str(e), 'error')
    
    def _update_status(self, message: str, status_type: str):
        """更新状态显示"""
        self.status_label.value = f'<span style="color: {self.STATUS_COLORS[status_type]};">{message}</span>'
    
    def get_config(self) -> Optional[Dict[str, str]]:
        """获取当前配置"""
        return self.current_config or None
    
    def display(self):
        """显示配置界面"""
        display(self.widget)
        # 触发平台变更事件来初始化界面状态
        self._on_platform_change(type('Change', (), {'new': self.platform_dropdown.value})())

def setup_chat_session():
    """创建并显示配置管理器，返回配置实例"""
    config_manager = APIConfigManager()
    config_manager.display()
    return config_manager

config_manager = setup_chat_session()

## 开始对话

> `ChatSession` 是一个非常基础的对话类。

In [3]:
class ChatSession:
    def __init__(self, api_key=None, base_url="https://api.deepseek.com", model="deepseek-chat", system_message="You are a helpful assistant."):
        """
        参数：
        - api_key (str): 平台的 API Key，默认从环境变量 `DEEPSEEK_API_KEY` 读取
        - base_url (str): API 请求地址，默认为 DeepSeek 官方平台
        - model (str): 模型名称（如 'deepseek-chat' 或 'deepseek-reasoner'），默认为 'deepseek-chat'
        - system_message (str): 系统消息，用于设定对话背景，默认为 'You are a helpful assistant.'
        """
        # 处理 API Key 优先级：显式传入 > 环境变量
        self.api_key = api_key or os.getenv("DEEPSEEK_API_KEY")
        if not self.api_key:
            raise ValueError("API Key 未提供，请通过参数传入或设置环境变量 DEEPSEEK_API_KEY")
        self.base_url = base_url
        
        # 初始化 OpenAI 客户端
        self.client = OpenAI(
            api_key=self.api_key,
            base_url=self.base_url,
        )
        
        self.model = model
        self.messages = [{'role': 'system', 'content': system_message}]

    def append_message(self, role, content):
        """
        添加一条对话消息

        参数:
        - role (str): 消息角色，为 'user' 或 'assistant'
        - content (str): 消息内容
        """
        self.messages.append({'role': role, 'content': content})

    def get_response(self, user_input, stream=False):
        """
        添加用户消息，调用 API 获取回复，并返回推理过程和回复内容

        参数：
        - user_input (str): 用户输入的消息
        - stream (bool): 是否启用流式输出，默认为 False

        返回：
        if stream=False:
            tuple: (reasoning_content, content)
            - reasoning_content (str|None): 推理过程，仅推理模型返回，聊天模型为 None
            - content (str): 模型的回复内容

        if stream=True:
            generator: 生成一系列 (reasoning_content, content) 元组
            - 对于推理过程: (reasoning_content, None)
            - 对于回复内容: (None, content)
            其中必定有一个值为 None，另一个包含当前数据块的实际内容
        """
        # 记录用户输入
        self.append_message('user', user_input)
        
        # 调用 API
        completion = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            stream=stream
        )
        
        if not stream:
            # 非流式输出
            content = completion.choices[0].message.content
            reasoning_content = getattr(completion.choices[0].message, 'reasoning_content', None)
            
            # 记录模型回复
            self.append_message('assistant', content)
            
            return reasoning_content, content
        else:
            # 流式输出，返回生成器
            return self._process_stream(completion)
    
    def _process_stream(self, completion):
        """
        处理流式输出的数据块

        参数：
        - completion: API 返回的流式输出对象

        返回：
        generator: 生成器对象，每次返回 (reasoning_content, content) 元组
        - 当收到推理过程时: yield (reasoning_content, None)
        - 当收到回复内容时: yield (None, content)
        """
        content = ""  # 用于存储完整回复
        
        for chunk in completion:
            delta = chunk.choices[0].delta
            # 处理推理过程（仅推理模型有）
            if getattr(delta, 'reasoning_content', None):
                yield delta.reasoning_content, None
            # 处理回复内容
            elif delta.content:
                content += delta.content  # 需要记录 content 维护对话历史
                yield None, delta.content
                
            # 如果是最后一个数据块（finish_reason 不为 None）
            if chunk.choices[0].finish_reason is not None:
                # 记录完整的模型回复 content
                self.append_message('assistant', content)
                break

> 一个更熟悉的写法（直接将输出逻辑放在类中）：
> 
> ```python
> def _process_stream(self, completion):
>     content = ""
>     
>     for chunk in completion:
>         delta = chunk.choices[0].delta
>         if getattr(delta, 'reasoning_content', None):
>             if delta.reasoning_content:
>                 print(reasoning_content, end='')
>             elif delta.content:
>                 content += delta.content
>                 print(delta.content, end='')
>                 
>     ...
> ```
>
> 此时会自动打印输出。

### 非流式输出

In [4]:
user_input = "Hello"
stream = False  # 非流式输出

# 更新配置
config = config_manager.get_config()
# print(config)
session = ChatSession(**config)

# 获取回复
reasoning, reply = session.get_response(user_input, stream=stream)
if reasoning:
    print(f"===== 推理过程 =====\n{reasoning}\n")
print(f"===== 模型回复 =====\nAI: {reply}\n")

===== 推理过程 =====
Okay, the user said "Hello." That's pretty straightforward. I should greet them back warmly. Let me respond with a friendly "Hello!" and ask how I can assist them today. Keeping it open-ended encourages them to ask for whatever they need.

===== 模型回复 =====
AI: Hello! How can I assist you today?



### 流式输出

In [5]:
user_input = "Hello"
stream = True  # 流式输出

# 更新配置
config = config_manager.get_config()
# print(config)
session = ChatSession(**config)

# 实时打印模型回复的增量内容，print() 设置 end=''实现不换行输出
for reasoning, reply in session.get_response(user_input, stream=stream):
    if reasoning:
        print(reasoning, end='')  
    else:
        print(reply, end='')

Okay, the user sent "Hello". I need to respond appropriately. Let me start by greeting them warmly. Maybe say something like, "Hello! How can I assist you today?" That sounds friendly and opens the door for them to ask for help. Make sure to keep it simple and welcoming without being overbearing.


Hello! How can I assist you today?